In [13]:
import angr
import sys

from angr.codenode import BlockNode
from angr.knowledge_plugins.functions.function import Function
from capstone.x86_const import X86_OP_REG, X86_OP_IMM

In [14]:
program = "test"
proj = angr.Project(program, load_options={"auto_load_libs": False})
main = proj.loader.find_symbol("main")
cfg = proj.analyses.CFGEmulated(context_sensitivity_level=0, starts=[main.rebased_addr], call_depth=0, normalize=True)
func = cfg.functions[main.rebased_addr]
graph = func.transition_graph

WARNING | 2019-02-27 19:09:43,419 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeff88 has an unspecified value; Generating an unconstrained value of 4 bytes.
WARNING | 2019-02-27 19:09:43,572 | angr.state_plugins.symbolic_memory | Memory address 0x7ffffffffff0000 has an unspecified value; Generating an unconstrained value of 8 bytes.
WARNING | 2019-02-27 19:09:43,781 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeff8c has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:43,835 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeff8d has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:43,886 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeff8e has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:43,941 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeff8f has 

WARNING | 2019-02-27 19:09:45,991 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffb4 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:46,048 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffb5 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:46,101 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffb6 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:46,157 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffb7 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:46,219 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffb8 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:46,276 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffb9 has 

WARNING | 2019-02-27 19:09:48,443 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffde has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:48,502 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffdf has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:48,555 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffe0 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:48,616 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffe1 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:48,672 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffe2 has an unspecified value; Generating an unconstrained value of 1 bytes.
WARNING | 2019-02-27 19:09:48,727 | angr.state_plugins.symbolic_memory | Memory address 0x7fffffffffeffe3 has 

In [15]:
def get_node(nodes, addr, fuzzy=False):
    if fuzzy:
        for node in nodes:
            if node.addr <= addr < node.addr + node.size:
                return node
        return None
    for node in nodes:
        if node.addr == addr:
            return node
    return None

In [16]:
def getsuccessors(graph, node):
    return list(graph.successors(node))

In [17]:
def haslog(succs):
    for succ in succs:
        if isinstance(succ, Function):
            if succ.name == '__afl_maybe_log':
                return True
    return False

In [18]:
def isIdinsn(insn):
    if insn.mnemonic != 'mov':
        return False
    if len(insn.operands) != 2:
        return False
    if insn.operands[0].type != X86_OP_REG:
        return False
    if insn.reg_name(insn.operands[0].value.reg) != 'rcx':
        return False
    if insn.operands[1].type != X86_OP_IMM:
        return False
    return True

def getId(block):
    for insn in reversed(block.capstone.insns):
        if isIdinsn(insn):
            return insn.operands[1].value.imm
    return 0

In [19]:
startAddr = main.rebased_addr
endAddr = 0x41a5e6

In [20]:
block_ids = dict()
edge_ids = dict()
def parse(graph, addr, prev_id, visit):
    if addr in visit:
        return
    visit.append(addr)
    cur_node = get_node(graph, addr)
    if isinstance(cur_node, Function):
        return
    succs = getsuccessors(graph, cur_node)
    if haslog(succs):
        block = proj.factory.block(cur_node.addr)
        cur_id = getId(block)
        block_ids[addr] = cur_id
        edge_ids[(prev_id, cur_id)] = (prev_id >> 1) ^ cur_id
        print("%x: %x -> %x Edge ID: %d" % (addr, prev_id, cur_id, (prev_id >> 1) ^ cur_id))
        for succ in succs:
            parse(graph, succ.addr, cur_id, visit)
    else:
        block_ids[addr] = prev_id
        for succ in succs:
            parse(graph, succ.addr, prev_id, visit)

In [21]:
visit = []
parse(graph, main.rebased_addr, 0, visit)

400800: 0 -> f459 Edge ID: 62553
400867: f459 -> f9c9 Edge ID: 33765
4008ba: f9c9 -> e493 Edge ID: 39031
402411: e493 -> faa Edge ID: 32227
4027fb: e493 -> 6343 Edge ID: 4362
402839: 6343 -> 9e2e Edge ID: 44943
40091f: e493 -> 11a0 Edge ID: 25577
400962: 11a0 -> ebdf Edge ID: 58127
4009a7: ebdf -> 25cf Edge ID: 20512
4009eb: 25cf -> 1439 Edge ID: 1758
400a2f: 1439 -> 8da7 Edge ID: 34747
400a73: 8da7 -> 2fb1 Edge ID: 26978
400ab7: 2fb1 -> b5e9 Edge ID: 41521
400afb: b5e9 -> fd68 Edge ID: 42908
400b3f: fd68 -> 57e Edge ID: 31690
400b83: 57e -> 147 Edge ID: 1016
400bc7: 147 -> fba0 Edge ID: 64259
400c0b: fba0 -> 902b Edge ID: 60923
400c4f: 902b -> 8218 Edge ID: 51725
400c93: 8218 -> c993 Edge ID: 34975
400cd7: c993 -> 48a4 Edge ID: 11373
400d1b: 48a4 -> adb7 Edge ID: 35301
400d5f: adb7 -> 9568 Edge ID: 50099
400da3: 9568 -> 2f2b Edge ID: 26015
400de7: 2f2b -> 5f80 Edge ID: 18453
40306f: 5f80 -> 109b Edge ID: 16219
400e2b: 5f80 -> f038 Edge ID: 57336
400e6f: f038 -> 82c6 Edge ID: 64218
402

In [10]:
sort_addr = sorted(block_ids.keys()) 

In [11]:
def getBlockid(addr):
    prev_addr = 0
    for each in sort_addr:
        if each > addr:
            break
        prev_addr = each
    return block_ids[prev_addr]

In [12]:
def getEdgeid(fromAddr, toAddr):
    from_id = getBlockid(fromAddr)
    if from_id == 0:
        print("invalid addr 0x%x" % fromAddr)
        return 0
    to_id = getBlockid(toAddr)
    if to_id == 0:
        print("invalid addr 0x%x" % toAddr)
        return 0
    if from_id == to_id:
        print("they are in the same block! %x-%x" % (fromAddr, toAddr))
        return 0
    if (from_id, to_id) in edge_ids:
        print("Found edge from %x to %x" % (from_id, to_id))
        return edge_ids[(from_id, to_id)]
    else:
        print("Error: can not find the tuple (%x, %x)" % (from_id, to_id))
    return 0

In [65]:
(getEdgeid(0x402b82, 0x4118c4))

Error: can not find the tuple (6a0, 4966)


0

In [70]:
((0xebe2 >> 1) ^ 0x2639)

21448

In [69]:
chr(0x62)

'b'